# Here i want to extract from lichess data features important for future modeling

So here i want to get data in format similar to that: 

columns = ( date, amount of mathes played in that date,ammont of matches played day earllier , amount of matches plyed for previous mounth, ranikng points fluctuation (day vs day before), ranking after last match that day, mean oponent vs my ranking diff that day ) and maybe some other columns that i will consider as important for ranking changes prediction. --> df2

And all of those for different times control (blitz, rapid, bullet)

and other data for winner prediction models, similar columns as without any chnages but some new cols like: time used in game, time for move and maybe some other --> df1

# To Be Continued

In [1]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context
import pandas as pd
import numpy as np
import arrow
pd.set_option('display.max_columns', None)

In [2]:
df = pd.read_csv("https://raw.githubusercontent.com/MikolajZaborowski/LichessData/main/Chess_data_basic.csv", index_col = "Unnamed: 0")

In [3]:
len(set(df["black_player_name"]+df["white_player_name"]))

4789

In [4]:
df.isna().sum()

id                     0
rated                  0
variant                0
speed                  0
createdAt              0
lastMoveAt             0
status                 0
winner               250
moves                  1
clock                  0
movesperplayer         0
white_player_name      0
white_ranking          0
black_player_name      0
black_ranking          0
dtype: int64

In [5]:
def convert_date_with_offset(date_str):
    return arrow.get(date_str).datetime

for i in ['createdAt', 'lastMoveAt']:
    df[i] = df[i].apply(convert_date_with_offset)
    
# Create Gametime column
df["Gametime"] = df["lastMoveAt"] - df["createdAt"]
# Handle Gametime column
df['Gametime'] = pd.to_timedelta(df['Gametime'])

rounding = lambda x: round(x, 1)

df['Gametime'] = df['Gametime'].dt.total_seconds().apply(rounding)
df['timepermove']=(df['Gametime']/df["movesperplayer"]).apply(rounding)

next think to do is taking insight from moves column, there is a lot of possibility but i will concentrate on:
1) cheking if side castle and if long or short castle
2) taking number of moves before castle ( if no castle - values set to number of moves +1 )
3) groping by opening (for this project i will take in count only first 6 moves - 3 moves per side )
4) number of checks in first 10 moves ( per side )

## Handle NA values

In [6]:
# Only timepermove colum had NA values (just one), expect winner, where NA == DRAW
df.dropna(subset=['timepermove'],inplace=True)

df = df.fillna("draw")

df.isna().sum()

id                   0
rated                0
variant              0
speed                0
createdAt            0
lastMoveAt           0
status               0
winner               0
moves                0
clock                0
movesperplayer       0
white_player_name    0
white_ranking        0
black_player_name    0
black_ranking        0
Gametime             0
timepermove          0
dtype: int64

# Checking for side castle 

kingside castle ( short ) is O-O in moves notation and queenside castle ( long ) is O-O-O

In [7]:
pd.options.mode.chained_assignment = None

In [8]:
def moves_column(df):
    df["moves"] = df["moves"].str.split(" ")

    df["white_moves"]=df["moves"].apply(lambda x: x[::2])
    df["black_moves"]=df["moves"].apply(lambda x: x[1::2])
    return df

df = moves_column(df)

In [9]:
def check_for_queen_or_king_castle(df):
    check_for_kingcastle = lambda x: 1 if "O-O" in x else 0
    check_for_queencastle = lambda x: 1 if "O-O-O" in x else 0
    df["white_kingcastle"] = df["white_moves"].apply(check_for_kingcastle)
    df["white_queencastle"] = df["white_moves"].apply(check_for_queencastle)
    df["black_kingcastle"] = df["black_moves"].apply(check_for_kingcastle)
    df["black_queencastle"] = df["black_moves"].apply(check_for_queencastle)
    return df

df = check_for_queen_or_king_castle(df)

# num of moves before castle 

In [10]:
def find_index_of_first_occurrence(lst):
    elements = ["O-O", "O-O-O"]
    for element in elements:
        if element in lst:
            return lst.index(element)
    return len(lst) + 1

df["White_moves_before_castle"] = df["white_moves"].apply(find_index_of_first_occurrence)
df["Black_moves_before_castle"] = df["black_moves"].apply(find_index_of_first_occurrence)

In [11]:
df.to_csv("to_visual.csv")

In [12]:
df.to_json("to_visual1.json")

## Grouping by opening

In [13]:
df["open4"] = df["moves"].apply(lambda x: x[:4])
df["open4"] = df["open4"].apply(lambda x: " ".join(x))

In [14]:
white_opens = df[df['white_player_name'] == 'EvilSaintPL']["open4"].value_counts().head(20)
black_opens = df[df['black_player_name'] == 'EvilSaintPL']["open4"].value_counts().head(20)

In [15]:
# openings_list = []
# def openings2( df ):
#     for index, row in df.iterrows():
#         if df["white_player_name"][index]=="EvilSaintPL" and df["open4"][index] in white_opens.index:
#             openings_list.append("White " + df["open4"][index])
#         elif  df["white_player_name"][index]!="EvilSaintPL" and df["open4"][index] in black_opens.index:
#             openings_list.append("Black " + df["open4"][index]) 
#         else:
#             openings_list.append("no opens")
    
    
    
#     df["opens1"]=result
#     return df

# df = openings2( df )

In [16]:

def iterable_opening_list_and_my_ranking( df ):
    
    openings_list = []
    rank_of_evilsaintpl_list = []
    evilsaintpl_won = []
    enemy_rank_list = []

    
    for index, row in df.iterrows():
        
        # OPENS1
        if df["white_player_name"][index]=="EvilSaintPL" and df["open4"][index] in white_opens.index:
            openings_list.append("White " + df["open4"][index])
        elif  df["white_player_name"][index]!="EvilSaintPL" and df["open4"][index] in black_opens.index:
            openings_list.append("Black " + df["open4"][index]) 
        else:
            openings_list.append("no opens")
        
        # MY RANK AND ENEMY RANK 
        if df["white_player_name"][index]=="EvilSaintPL":
            rank_of_evilsaintpl_list.append(df["white_ranking"][index])
            enemy_rank_list.append(df["black_ranking"][index])
        elif df["black_player_name"][index]=="EvilSaintPL":
            rank_of_evilsaintpl_list.append(df["black_ranking"][index])
            enemy_rank_list.append(df["white_ranking"][index])
        else:
            print(index)
            
        # I WON 
        if df["winner"][index] == 'draw':
            evilsaintpl_won.append(0)
        elif df["white_player_name"][index]=="EvilSaintPL" and df["winner"][index]=='white':
            evilsaintpl_won.append(1)
        elif  df["black_player_name"][index]=="EvilSaintPL" and df["winner"][index]=='black':
            evilsaintpl_won.append(1)
        else:
            evilsaintpl_won.append(-1)

    df["opens1"]= openings_list
    df["my_rank"]= rank_of_evilsaintpl_list
    df['i_won'] = evilsaintpl_won
    df['enemy_rank'] = enemy_rank_list
    
    # NEW RANKS FE
    df['rank_difference'] = df['my_rank'] - df['enemy_rank']
    df['is_higher'] = (df['my_rank'] > df['enemy_rank'])


    return df

df = iterable_opening_list_and_my_ranking(df)

In [17]:
df.head(1)

,id,rated,variant,speed,createdAt,lastMoveAt,status,winner,moves,clock,movesperplayer,white_player_name,white_ranking,black_player_name,black_ranking,Gametime,timepermove,white_moves,black_moves,white_kingcastle,white_queencastle,black_kingcastle,black_queencastle,White_moves_before_castle,Black_moves_before_castle,open4,opens1,my_rank,i_won,enemy_rank,rank_difference,is_higher
0,O83cQxfo,True,standard,blitz,2023-06-28 02:33:43.673000+02:00,2023-06-28 02:44:00.722000+02:00,outoftime,white,"[e4, e5, Nf3, Nc6, Bc4, Nf6, d4, exd4, e5, d5,...",300,70,EvilSaintPL,1753,Leosammie,1735,617.0,8.8,"[e4, Nf3, Bc4, d4, e5, exd6, O-O, Na3, Nb5, Nb...","[e5, Nc6, Nf6, exd4, d5, Qxd6, Be7, Bg4, Qd7, ...",1,0,0,0,6,71,e4 e5 Nf3 Nc6,White e4 e5 Nf3 Nc6,1753,1,1735,18,True


In [18]:
def clear_dataframe(df):
    temp = pd.get_dummies(df["opens1"])
    df = pd.concat([df,temp], axis=1)
    
    df.drop("status",axis=1, inplace=True)
    df.drop(labels = ["opens1"], axis=1, inplace=True)
#    df.drop("id",axis=1,inplace=True)
    
    df = df[df["rated"]==True]
    df = df[df["white_player_name"]!="YoungCapitan"]
    df = df[df["black_player_name"]!="YoungCapitan"]
    df.drop("variant",axis=1,inplace=True)
    
    return df

df = clear_dataframe(df)

In [19]:
df.head(1)

,id,rated,speed,createdAt,lastMoveAt,winner,moves,clock,movesperplayer,white_player_name,white_ranking,black_player_name,black_ranking,Gametime,timepermove,white_moves,black_moves,white_kingcastle,white_queencastle,black_kingcastle,black_queencastle,White_moves_before_castle,Black_moves_before_castle,open4,my_rank,i_won,enemy_rank,rank_difference,is_higher,Black c4 e5 Nc3 Nf6,Black d4 Nf6 Bf4 e6,Black d4 Nf6 Nf3 e6,Black d4 Nf6 c4 e6,Black d4 d5 Bf4 Nf6,Black d4 d5 c4 c6,Black e4 e5 Bc4 Nc6,Black e4 e5 Bc4 Nf6,Black e4 e5 Nc3 Nf6,Black e4 e5 Nf3 Nc6,Black e4 e5 Nf3 Nf6,Black e4 e5 d4 exd4,Black e4 e5 f4 d5,Black e4 e5 f4 exf4,Black e4 e6 Bc4 d5,Black e4 e6 Nc3 d5,Black e4 e6 Nf3 d5,Black e4 e6 d4 d5,Black e4 e6 e5 c5,Black g3 e5 Bg2 Nf6,White e4 c5 Nf3 Nc6,White e4 c5 Nf3 d6,White e4 c5 Nf3 e6,White e4 c6 Nf3 d5,White e4 c6 d4 d5,White e4 d5 Nf3 dxe4,White e4 d5 exd5 Nf6,White e4 d5 exd5 Qxd5,White e4 e5 Nc3 Nc6,White e4 e5 Nc3 Nf6,White e4 e5 Nc3 d6,White e4 e5 Nf3 Bc5,White e4 e5 Nf3 Nc6,White e4 e5 Nf3 Nf6,White e4 e5 Nf3 Qf6,White e4 e5 Nf3 d5,White e4 e5 Nf3 d6,White e4 e6 Nf3 d5,White e4 e6 d4 d5,White e4 g6 d4 Bg7,no opens
0,O83cQxfo,True,blitz,2023-06-28 02:33:43.673000+02:00,2023-06-28 02:44:00.722000+02:00,white,"[e4, e5, Nf3, Nc6, Bc4, Nf6, d4, exd4, e5, d5,...",300,70,EvilSaintPL,1753,Leosammie,1735,617.0,8.8,"[e4, Nf3, Bc4, d4, e5, exd6, O-O, Na3, Nb5, Nb...","[e5, Nc6, Nf6, exd4, d5, Qxd6, Be7, Bg4, Qd7, ...",1,0,0,0,6,71,e4 e5 Nf3 Nc6,1753,1,1735,18,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False


In [20]:
def results(df):
    game_result = pd.get_dummies(df["i_won"])
    game_result.columns=["Defeat","Draw","Win"]
    df = pd.concat([df, game_result], axis=1)
    df.drop("i_won",axis=1, inplace=True)
    return df

In [21]:
# df = results(df)

In [22]:
df["i_won"]=df["i_won"]+1

In [23]:
df.shape

(4880, 70)

In [24]:
def rankdiff(df):
    rank_resu=list()
    for index, row in df.iterrows():
        print(index)
        if index==0:
            rank_resu.append(1759)
        else:
            rank_resu.append(df["my_rank"][index-1])
    df["rank_after_game"]=rank_resu
    return df

In [25]:
df.speed.unique()

array(['blitz', 'classical', 'bullet', 'rapid'], dtype=object)

In [26]:
def rankdifftest(df):
    blitz = df[df["speed"]=="blitz"].reset_index(drop=True)
    classical = df[df["speed"]=="classical"].reset_index(drop=True)
    bullet = df[df["speed"]=="bullet"].reset_index(drop=True)
    rapid = df[df["speed"]=="rapid"].reset_index(drop=True)
    
    for speed in [blitz, classical, bullet, rapid]:
        rank_resu=list()
        for index, row in speed.iterrows():
            if index==0:
                rank_resu.append(speed["my_rank"][index])
            else:
                rank_resu.append(speed["my_rank"][index-1])
        speed["rank_after_game"]=rank_resu
        speed["my_rank_diff"]=speed["rank_after_game"]-speed["my_rank"]
    return pd.concat([blitz, classical, bullet, rapid])
        
    
    

In [27]:
df = rankdifftest(df)

In [28]:
df.shape

(4880, 72)

In [29]:
df.tail(2)

,id,rated,speed,createdAt,lastMoveAt,winner,moves,clock,movesperplayer,white_player_name,white_ranking,black_player_name,black_ranking,Gametime,timepermove,white_moves,black_moves,white_kingcastle,white_queencastle,black_kingcastle,black_queencastle,White_moves_before_castle,Black_moves_before_castle,open4,my_rank,i_won,enemy_rank,rank_difference,is_higher,Black c4 e5 Nc3 Nf6,Black d4 Nf6 Bf4 e6,Black d4 Nf6 Nf3 e6,Black d4 Nf6 c4 e6,Black d4 d5 Bf4 Nf6,Black d4 d5 c4 c6,Black e4 e5 Bc4 Nc6,Black e4 e5 Bc4 Nf6,Black e4 e5 Nc3 Nf6,Black e4 e5 Nf3 Nc6,Black e4 e5 Nf3 Nf6,Black e4 e5 d4 exd4,Black e4 e5 f4 d5,Black e4 e5 f4 exf4,Black e4 e6 Bc4 d5,Black e4 e6 Nc3 d5,Black e4 e6 Nf3 d5,Black e4 e6 d4 d5,Black e4 e6 e5 c5,Black g3 e5 Bg2 Nf6,White e4 c5 Nf3 Nc6,White e4 c5 Nf3 d6,White e4 c5 Nf3 e6,White e4 c6 Nf3 d5,White e4 c6 d4 d5,White e4 d5 Nf3 dxe4,White e4 d5 exd5 Nf6,White e4 d5 exd5 Qxd5,White e4 e5 Nc3 Nc6,White e4 e5 Nc3 Nf6,White e4 e5 Nc3 d6,White e4 e5 Nf3 Bc5,White e4 e5 Nf3 Nc6,White e4 e5 Nf3 Nf6,White e4 e5 Nf3 Qf6,White e4 e5 Nf3 d5,White e4 e5 Nf3 d6,White e4 e6 Nf3 d5,White e4 e6 d4 d5,White e4 g6 d4 Bg7,no opens,rank_after_game,my_rank_diff
333,ehegL7CY,True,rapid,2019-12-03 17:13:39.170000+01:00,2019-12-03 17:18:40.291000+01:00,white,"[e4, e5, Nf3, Nf6, Bc4, Nc6, Nc3, Bb4, Qe2, Bx...",600,13,Keeva_the_Slayer,1668,EvilSaintPL,1685,301.1,23.2,"[e4, Nf3, Bc4, Nc3, Qe2, dxc3, exd5, Nxe5, Qxe...","[e5, Nf6, Nc6, Bb4, Bxc3, d5, Nxd5, Nxe5, Kf8,...",1,0,0,0,11,14,e4 e5 Nf3 Nf6,1685,0,1668,17,True,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,1575,-110
334,nrBACFKZ,True,rapid,2019-12-03 17:10:43.213000+01:00,2019-12-03 17:13:24.831000+01:00,white,"[e4, c5, Nc3, e6, d4, cxd4, Qxd4, Nc6, Qd1, b6...",600,12,EvilSaintPL,1500,Zabbadoo,1502,161.6,13.5,"[e4, Nc3, d4, Qxd4, Qd1, Nf3, g3, Bg2, O-O, Bf...","[c5, e6, cxd4, Nc6, b6, Be7, d6, Nf6, O-O, Qc7...",1,0,1,0,8,8,e4 c5 Nc3 e6,1500,2,1502,-2,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,1685,185


In [30]:
df.columns

Index(['id', 'rated', 'speed', 'createdAt', 'lastMoveAt', 'winner', 'moves',
       'clock', 'movesperplayer', 'white_player_name', 'white_ranking',
       'black_player_name', 'black_ranking', 'Gametime', 'timepermove',
       'white_moves', 'black_moves', 'white_kingcastle', 'white_queencastle',
       'black_kingcastle', 'black_queencastle', 'White_moves_before_castle',
       'Black_moves_before_castle', 'open4', 'my_rank', 'i_won', 'enemy_rank',
       'rank_difference', 'is_higher', 'Black c4 e5 Nc3 Nf6',
       'Black d4 Nf6 Bf4 e6', 'Black d4 Nf6 Nf3 e6', 'Black d4 Nf6 c4 e6',
       'Black d4 d5 Bf4 Nf6', 'Black d4 d5 c4 c6', 'Black e4 e5 Bc4 Nc6',
       'Black e4 e5 Bc4 Nf6', 'Black e4 e5 Nc3 Nf6', 'Black e4 e5 Nf3 Nc6',
       'Black e4 e5 Nf3 Nf6', 'Black e4 e5 d4 exd4', 'Black e4 e5 f4 d5',
       'Black e4 e5 f4 exf4', 'Black e4 e6 Bc4 d5', 'Black e4 e6 Nc3 d5',
       'Black e4 e6 Nf3 d5', 'Black e4 e6 d4 d5', 'Black e4 e6 e5 c5',
       'Black g3 e5 Bg2 Nf6', 'White e4

In [36]:
final_all = df[[ 'i_won','my_rank', 'enemy_rank','rank_difference', 'is_higher' ,'clock', 'movesperplayer', 'Gametime', 'timepermove','white_kingcastle', 'white_queencastle',
       'black_kingcastle', 'black_queencastle', 'White_moves_before_castle', 'Black_moves_before_castle', 'no opens', 'Black c4 e5 Nc3 Nf6',
       'Black d4 Nf6 Bf4 e6', 'Black d4 Nf6 Nf3 e6', 'Black d4 Nf6 c4 e6',
       'Black d4 d5 Bf4 Nf6', 'Black d4 d5 c4 c6', 'Black e4 e5 Bc4 Nc6',
       'Black e4 e5 Bc4 Nf6', 'Black e4 e5 Nc3 Nf6', 'Black e4 e5 Nf3 Nc6',
       'Black e4 e5 Nf3 Nf6', 'Black e4 e5 d4 exd4', 'Black e4 e5 f4 d5',
       'Black e4 e5 f4 exf4', 'Black e4 e6 Bc4 d5', 'Black e4 e6 Nc3 d5',
       'Black e4 e6 Nf3 d5', 'Black e4 e6 d4 d5', 'Black e4 e6 e5 c5',
       'Black g3 e5 Bg2 Nf6', 'White e4 c5 Nf3 Nc6', 'White e4 c5 Nf3 d6',
       'White e4 c5 Nf3 e6', 'White e4 c6 Nf3 d5', 'White e4 c6 d4 d5',
       'White e4 d5 Nf3 dxe4', 'White e4 d5 exd5 Nf6', 'White e4 d5 exd5 Qxd5',
       'White e4 e5 Nc3 Nc6', 'White e4 e5 Nc3 Nf6', 'White e4 e5 Nc3 d6',
       'White e4 e5 Nf3 Bc5', 'White e4 e5 Nf3 Nc6', 'White e4 e5 Nf3 Nf6',
       'White e4 e5 Nf3 Qf6', 'White e4 e5 Nf3 d5', 'White e4 e5 Nf3 d6',
       'White e4 e6 Nf3 d5', 'White e4 e6 d4 d5', 'White e4 g6 d4 Bg7'  ]]

blitz = df[df["speed"]=="blitz"][[ 'i_won','my_rank', 'enemy_rank','rank_difference', 'is_higher', 'movesperplayer', 'Gametime', 'timepermove','white_kingcastle', 'white_queencastle',
       'black_kingcastle', 'black_queencastle', 'White_moves_before_castle', 'Black_moves_before_castle', 'no opens', 'Black c4 e5 Nc3 Nf6',
       'Black d4 Nf6 Bf4 e6', 'Black d4 Nf6 Nf3 e6', 'Black d4 Nf6 c4 e6',
       'Black d4 d5 Bf4 Nf6', 'Black d4 d5 c4 c6', 'Black e4 e5 Bc4 Nc6',
       'Black e4 e5 Bc4 Nf6', 'Black e4 e5 Nc3 Nf6', 'Black e4 e5 Nf3 Nc6',
       'Black e4 e5 Nf3 Nf6', 'Black e4 e5 d4 exd4', 'Black e4 e5 f4 d5',
       'Black e4 e5 f4 exf4', 'Black e4 e6 Bc4 d5', 'Black e4 e6 Nc3 d5',
       'Black e4 e6 Nf3 d5', 'Black e4 e6 d4 d5', 'Black e4 e6 e5 c5',
       'Black g3 e5 Bg2 Nf6', 'White e4 c5 Nf3 Nc6', 'White e4 c5 Nf3 d6',
       'White e4 c5 Nf3 e6', 'White e4 c6 Nf3 d5', 'White e4 c6 d4 d5',
       'White e4 d5 Nf3 dxe4', 'White e4 d5 exd5 Nf6', 'White e4 d5 exd5 Qxd5',
       'White e4 e5 Nc3 Nc6', 'White e4 e5 Nc3 Nf6', 'White e4 e5 Nc3 d6',
       'White e4 e5 Nf3 Bc5', 'White e4 e5 Nf3 Nc6', 'White e4 e5 Nf3 Nf6',
       'White e4 e5 Nf3 Qf6', 'White e4 e5 Nf3 d5', 'White e4 e5 Nf3 d6',
       'White e4 e6 Nf3 d5', 'White e4 e6 d4 d5', 'White e4 g6 d4 Bg7'  ]]

classical = df[df["speed"]=="classical"][[ 'i_won','my_rank', 'enemy_rank','rank_difference', 'is_higher', 'movesperplayer', 'Gametime', 'timepermove','white_kingcastle', 'white_queencastle',
       'black_kingcastle', 'black_queencastle', 'White_moves_before_castle', 'Black_moves_before_castle', 'no opens', 'Black c4 e5 Nc3 Nf6',
       'Black d4 Nf6 Bf4 e6', 'Black d4 Nf6 Nf3 e6', 'Black d4 Nf6 c4 e6',
       'Black d4 d5 Bf4 Nf6', 'Black d4 d5 c4 c6', 'Black e4 e5 Bc4 Nc6',
       'Black e4 e5 Bc4 Nf6', 'Black e4 e5 Nc3 Nf6', 'Black e4 e5 Nf3 Nc6',
       'Black e4 e5 Nf3 Nf6', 'Black e4 e5 d4 exd4', 'Black e4 e5 f4 d5',
       'Black e4 e5 f4 exf4', 'Black e4 e6 Bc4 d5', 'Black e4 e6 Nc3 d5',
       'Black e4 e6 Nf3 d5', 'Black e4 e6 d4 d5', 'Black e4 e6 e5 c5',
       'Black g3 e5 Bg2 Nf6', 'White e4 c5 Nf3 Nc6', 'White e4 c5 Nf3 d6',
       'White e4 c5 Nf3 e6', 'White e4 c6 Nf3 d5', 'White e4 c6 d4 d5',
       'White e4 d5 Nf3 dxe4', 'White e4 d5 exd5 Nf6', 'White e4 d5 exd5 Qxd5',
       'White e4 e5 Nc3 Nc6', 'White e4 e5 Nc3 Nf6', 'White e4 e5 Nc3 d6',
       'White e4 e5 Nf3 Bc5', 'White e4 e5 Nf3 Nc6', 'White e4 e5 Nf3 Nf6',
       'White e4 e5 Nf3 Qf6', 'White e4 e5 Nf3 d5', 'White e4 e5 Nf3 d6',
       'White e4 e6 Nf3 d5', 'White e4 e6 d4 d5', 'White e4 g6 d4 Bg7'  ]]

bullet = df[df["speed"]=="bullet"][[ 'i_won','my_rank', 'enemy_rank','rank_difference', 'is_higher', 'movesperplayer', 'Gametime', 'timepermove','white_kingcastle', 'white_queencastle',
       'black_kingcastle', 'black_queencastle', 'White_moves_before_castle', 'Black_moves_before_castle', 'no opens', 'Black c4 e5 Nc3 Nf6',
       'Black d4 Nf6 Bf4 e6', 'Black d4 Nf6 Nf3 e6', 'Black d4 Nf6 c4 e6',
       'Black d4 d5 Bf4 Nf6', 'Black d4 d5 c4 c6', 'Black e4 e5 Bc4 Nc6',
       'Black e4 e5 Bc4 Nf6', 'Black e4 e5 Nc3 Nf6', 'Black e4 e5 Nf3 Nc6',
       'Black e4 e5 Nf3 Nf6', 'Black e4 e5 d4 exd4', 'Black e4 e5 f4 d5',
       'Black e4 e5 f4 exf4', 'Black e4 e6 Bc4 d5', 'Black e4 e6 Nc3 d5',
       'Black e4 e6 Nf3 d5', 'Black e4 e6 d4 d5', 'Black e4 e6 e5 c5',
       'Black g3 e5 Bg2 Nf6', 'White e4 c5 Nf3 Nc6', 'White e4 c5 Nf3 d6',
       'White e4 c5 Nf3 e6', 'White e4 c6 Nf3 d5', 'White e4 c6 d4 d5',
       'White e4 d5 Nf3 dxe4', 'White e4 d5 exd5 Nf6', 'White e4 d5 exd5 Qxd5',
       'White e4 e5 Nc3 Nc6', 'White e4 e5 Nc3 Nf6', 'White e4 e5 Nc3 d6',
       'White e4 e5 Nf3 Bc5', 'White e4 e5 Nf3 Nc6', 'White e4 e5 Nf3 Nf6',
       'White e4 e5 Nf3 Qf6', 'White e4 e5 Nf3 d5', 'White e4 e5 Nf3 d6',
       'White e4 e6 Nf3 d5', 'White e4 e6 d4 d5', 'White e4 g6 d4 Bg7'  ]]

rapid = df[df["speed"]=="rapid"][[ 'i_won','my_rank', 'enemy_rank','rank_difference', 'is_higher', 'movesperplayer', 'Gametime', 'timepermove','white_kingcastle', 'white_queencastle',
       'black_kingcastle', 'black_queencastle', 'White_moves_before_castle', 'Black_moves_before_castle', 'no opens', 'Black c4 e5 Nc3 Nf6',
       'Black d4 Nf6 Bf4 e6', 'Black d4 Nf6 Nf3 e6', 'Black d4 Nf6 c4 e6',
       'Black d4 d5 Bf4 Nf6', 'Black d4 d5 c4 c6', 'Black e4 e5 Bc4 Nc6',
       'Black e4 e5 Bc4 Nf6', 'Black e4 e5 Nc3 Nf6', 'Black e4 e5 Nf3 Nc6',
       'Black e4 e5 Nf3 Nf6', 'Black e4 e5 d4 exd4', 'Black e4 e5 f4 d5',
       'Black e4 e5 f4 exf4', 'Black e4 e6 Bc4 d5', 'Black e4 e6 Nc3 d5',
       'Black e4 e6 Nf3 d5', 'Black e4 e6 d4 d5', 'Black e4 e6 e5 c5',
       'Black g3 e5 Bg2 Nf6', 'White e4 c5 Nf3 Nc6', 'White e4 c5 Nf3 d6',
       'White e4 c5 Nf3 e6', 'White e4 c6 Nf3 d5', 'White e4 c6 d4 d5',
       'White e4 d5 Nf3 dxe4', 'White e4 d5 exd5 Nf6', 'White e4 d5 exd5 Qxd5',
       'White e4 e5 Nc3 Nc6', 'White e4 e5 Nc3 Nf6', 'White e4 e5 Nc3 d6',
       'White e4 e5 Nf3 Bc5', 'White e4 e5 Nf3 Nc6', 'White e4 e5 Nf3 Nf6',
       'White e4 e5 Nf3 Qf6', 'White e4 e5 Nf3 d5', 'White e4 e5 Nf3 d6',
       'White e4 e6 Nf3 d5', 'White e4 e6 d4 d5', 'White e4 g6 d4 Bg7'  ]]

In [37]:
print(f"blitz : {blitz.shape},\nclassic: {classical.shape},\nbullet: {bullet.shape},\nrapid: {rapid.shape},\nfull: {final_all.shape}")

blitz : (3417, 55),
classic: (153, 55),
bullet: (975, 55),
rapid: (335, 55),
full: (4880, 56)


In [38]:
blitz.head(2)

,i_won,my_rank,enemy_rank,rank_difference,is_higher,movesperplayer,Gametime,timepermove,white_kingcastle,white_queencastle,black_kingcastle,black_queencastle,White_moves_before_castle,Black_moves_before_castle,no opens,Black c4 e5 Nc3 Nf6,Black d4 Nf6 Bf4 e6,Black d4 Nf6 Nf3 e6,Black d4 Nf6 c4 e6,Black d4 d5 Bf4 Nf6,Black d4 d5 c4 c6,Black e4 e5 Bc4 Nc6,Black e4 e5 Bc4 Nf6,Black e4 e5 Nc3 Nf6,Black e4 e5 Nf3 Nc6,Black e4 e5 Nf3 Nf6,Black e4 e5 d4 exd4,Black e4 e5 f4 d5,Black e4 e5 f4 exf4,Black e4 e6 Bc4 d5,Black e4 e6 Nc3 d5,Black e4 e6 Nf3 d5,Black e4 e6 d4 d5,Black e4 e6 e5 c5,Black g3 e5 Bg2 Nf6,White e4 c5 Nf3 Nc6,White e4 c5 Nf3 d6,White e4 c5 Nf3 e6,White e4 c6 Nf3 d5,White e4 c6 d4 d5,White e4 d5 Nf3 dxe4,White e4 d5 exd5 Nf6,White e4 d5 exd5 Qxd5,White e4 e5 Nc3 Nc6,White e4 e5 Nc3 Nf6,White e4 e5 Nc3 d6,White e4 e5 Nf3 Bc5,White e4 e5 Nf3 Nc6,White e4 e5 Nf3 Nf6,White e4 e5 Nf3 Qf6,White e4 e5 Nf3 d5,White e4 e5 Nf3 d6,White e4 e6 Nf3 d5,White e4 e6 d4 d5,White e4 g6 d4 Bg7
0,2,1753,1735,18,True,70,617.0,8.8,1,0,0,0,6,71,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False
1,0,1758,1801,-43,False,18,105.1,5.8,1,0,0,1,17,14,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


In [39]:
rapid.head(2)

,i_won,my_rank,enemy_rank,rank_difference,is_higher,movesperplayer,Gametime,timepermove,white_kingcastle,white_queencastle,black_kingcastle,black_queencastle,White_moves_before_castle,Black_moves_before_castle,no opens,Black c4 e5 Nc3 Nf6,Black d4 Nf6 Bf4 e6,Black d4 Nf6 Nf3 e6,Black d4 Nf6 c4 e6,Black d4 d5 Bf4 Nf6,Black d4 d5 c4 c6,Black e4 e5 Bc4 Nc6,Black e4 e5 Bc4 Nf6,Black e4 e5 Nc3 Nf6,Black e4 e5 Nf3 Nc6,Black e4 e5 Nf3 Nf6,Black e4 e5 d4 exd4,Black e4 e5 f4 d5,Black e4 e5 f4 exf4,Black e4 e6 Bc4 d5,Black e4 e6 Nc3 d5,Black e4 e6 Nf3 d5,Black e4 e6 d4 d5,Black e4 e6 e5 c5,Black g3 e5 Bg2 Nf6,White e4 c5 Nf3 Nc6,White e4 c5 Nf3 d6,White e4 c5 Nf3 e6,White e4 c6 Nf3 d5,White e4 c6 d4 d5,White e4 d5 Nf3 dxe4,White e4 d5 exd5 Nf6,White e4 d5 exd5 Qxd5,White e4 e5 Nc3 Nc6,White e4 e5 Nc3 Nf6,White e4 e5 Nc3 d6,White e4 e5 Nf3 Bc5,White e4 e5 Nf3 Nc6,White e4 e5 Nf3 Nf6,White e4 e5 Nf3 Qf6,White e4 e5 Nf3 d5,White e4 e5 Nf3 d6,White e4 e6 Nf3 d5,White e4 e6 d4 d5,White e4 g6 d4 Bg7
0,0,1790,1785,5,True,62,1079.0,17.4,1,0,1,0,10,9,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False
1,2,1777,1783,-6,False,37,845.4,22.8,0,1,1,0,10,8,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


In [61]:
blitz = blitz[~blitz["timepermove"].isin([np.inf, -np.inf])]

In [85]:
# column definiton
"""
i_won - 0 -> I lost, 1-Draw, 2-I_Won
is_higher -> is my rank higher than my enemy
timepermove -> Gametime / movesperplayer
White_moves_before_castle -> amount of white moves before castle
no opens - no specyfic opening typed played by evilsainpl
"""

'\ni_won - 0 -> I lost, 1-Draw, 2-I_Won\nis_higher -> is my rank higher than my enemy\ntimepermove -> Gametime / movesperplayer\nWhite_moves_before_castle -> amount of white moves before castle\nno opens - no specyfic opening typed played by evilsainpl\n'